In [37]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import pandas as pd
import selenium
from webdriver_manager.chrome import ChromeDriverManager
import chromedriver_autoinstaller
import requests
import os
import glob
import zipfile
import time
from pathlib import Path

In [38]:
def get_file_dataframe():
    
    list_of_files = glob.glob(f'{Path.home()}/Downloads/*.xls') # * means all if need specific format then *.csv
    #print(list_of_files)
    latest_file = max(list_of_files, key=os.path.getctime)
    filename = latest_file
    #print(filename)
    df = pd.read_excel(filename,sheet_name = 0)
    return df

def delete_files_used(times):
    
    for i in range(times):
        
        list_of_files = glob.glob(f'{Path.home()}/Downloads/*.xls') # * means all if need specific format then *.csv
        latest_file = max(list_of_files, key=os.path.getctime)
        os.remove(latest_file)
        #print('ok')
        
    return 

def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 20:
        time.sleep(1)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
        seconds += 1
    return seconds;

In [39]:
# setup-selenium

chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path

driver = webdriver.Chrome()
driver.maximize_window()

In [40]:
url = 'https://www.fao.org/worldfoodsituation/foodpricesindex/en/'
driver.get(url)

In [41]:
dataset_xpath = '//*[@id="c596897"]/table[2]/tbody/tr/td[1]/p/a[1]'
driver.find_element("xpath",dataset_xpath).click()
download_wait(f'{Path.home()}/Downloads/');

In [42]:
driver.close()
download_wait(f'{Path.home()}/Downloads/')
df = get_file_dataframe()

n_times = 1

delete_files_used(n_times);

df = df.drop(columns = df.columns[6:]).dropna()
df

,FAO Food Price Index,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
1,Date,Food Price Index,Meat,Dairy,Cereals,Oils
3,1990-01-01 00:00:00,64.145081,73.374551,53.503031,64.140607,44.587672
4,1990-02-01 00:00:00,64.45356,75.956526,52.218634,62.222377,44.500514
5,1990-03-01 00:00:00,63.787064,77.805859,41.367123,61.259884,45.745434
6,1990-04-01 00:00:00,65.766754,80.442049,48.427068,62.820731,44.017094
...,...,...,...,...,...,...
395,2022-09-01 00:00:00,136.041941,120.276505,142.693528,147.915084,152.572785
396,2022-10-01 00:00:00,135.399247,116.833255,139.252815,152.345051,151.284973
397,2022-11-01 00:00:00,134.738443,114.628969,137.385241,150.108623,154.707117
398,2022-12-01 00:00:00,132.238307,113.735635,138.169792,147.254006,144.600299


In [43]:
df.columns = df.iloc[0]
df = df.drop(df.index[0])

In [44]:
df.columns[1:]

Index(['Food Price Index', 'Meat', 'Dairy', 'Cereals', 'Oils'], dtype='object', name=1)

In [46]:
df

1,Date,Food Price Index,Meat,Dairy,Cereals,Oils
3,1990-01-01 00:00:00,64.145081,73.374551,53.503031,64.140607,44.587672
4,1990-02-01 00:00:00,64.45356,75.956526,52.218634,62.222377,44.500514
5,1990-03-01 00:00:00,63.787064,77.805859,41.367123,61.259884,45.745434
6,1990-04-01 00:00:00,65.766754,80.442049,48.427068,62.820731,44.017094
7,1990-05-01 00:00:00,64.378874,80.994505,39.189651,61.982826,45.499631
...,...,...,...,...,...,...
395,2022-09-01 00:00:00,136.041941,120.276505,142.693528,147.915084,152.572785
396,2022-10-01 00:00:00,135.399247,116.833255,139.252815,152.345051,151.284973
397,2022-11-01 00:00:00,134.738443,114.628969,137.385241,150.108623,154.707117
398,2022-12-01 00:00:00,132.238307,113.735635,138.169792,147.254006,144.600299


In [49]:
base_path = 'C:\\Users\\guilh\\TM\\var_comodity\\JSONS\\FAO\\FOODINDEX\\'

for i in df.columns[1:]:
    
    name = i
    #print(name)
    try:
        df_filter = df[['Date',name]]
        #print('aqui')
    except:
        pass
    
    name_var = f'{name}_FAO_gbl_und_y1_.json'
    folder_path = base_path+name_var
    df = df_filter
    if not df.empty:
    #        print(df.empty)
        df.to_json(folder_path,indent =1)
    else:
        pass
